**Python task 2**

In [4]:
import pandas as pd
df3=pd.read_csv('/content/sample_data/dataset-3.csv')
df3

,id_start,id_end,distance
0,1001400,1001402,9.7
1,1001402,1001404,20.2
2,1001404,1001406,16.0
3,1001406,1001408,21.7
4,1001408,1001410,11.1
5,1001410,1001412,15.6
6,1001412,1001414,18.2
7,1001414,1001416,13.2
8,1001416,1001418,13.6
9,1001418,1001420,12.9


In [5]:
from geopy.distance import geodesic


In [30]:
def calculate_distance_matrix(df):

    unique_ids = sorted(set(df['id_start'].unique()) | set(df['id_end'].unique()))


    distance_matrix = pd.DataFrame(index=unique_ids, columns=unique_ids, dtype=float)


    for row in df.itertuples(index=False):

        distance_matrix.loc[row.id_start, row.id_end] = row.distance
        distance_matrix.loc[row.id_end, row.id_start] = row.distance


    distance_matrix.values[[range(len(unique_ids))]*2] = 0.0

    return distance_matrix


In [31]:
distance_matrix_df = calculate_distance_matrix(df3)
print("Distance Matrix:")
print(distance_matrix_df)

Distance Matrix:
         1001400  1001402  1001404  1001406  1001408  1001410  1001412  \
1001400      0.0      0.0      0.0      0.0      0.0      0.0      0.0   
1001402      0.0      0.0      0.0      0.0      0.0      0.0      0.0   
1001404      0.0      0.0      0.0      0.0      0.0      0.0      0.0   
1001406      0.0      0.0      0.0      0.0      0.0      0.0      0.0   
1001408      0.0      0.0      0.0      0.0      0.0      0.0      0.0   
1001410      0.0      0.0      0.0      0.0      0.0      0.0      0.0   
1001412      0.0      0.0      0.0      0.0      0.0      0.0      0.0   
1001414      0.0      0.0      0.0      0.0      0.0      0.0      0.0   
1001416      0.0      0.0      0.0      0.0      0.0      0.0      0.0   
1001418      0.0      0.0      0.0      0.0      0.0      0.0      0.0   
1001420      0.0      0.0      0.0      0.0      0.0      0.0      0.0   
1001422      0.0      0.0      0.0      0.0      0.0      0.0      0.0   
1001424      0.0     

In [28]:
def unroll_distance_matrix(distance_matrix):

    ids = distance_matrix.index


    id_start_list, id_end_list, distance_list = [], [], []


    for id_start in ids:
        for id_end in ids:

            if id_start == id_end:
                continue


            id_start_list.append(id_start)
            id_end_list.append(id_end)
            distance_list.append(distance_matrix.loc[id_start, id_end])


    unrolled_df = pd.DataFrame({
        'id_start': id_start_list,
        'id_end': id_end_list,
        'distance': distance_list
    })

    return unrolled_df

In [29]:
result_matrix = calculate_distance_matrix(df3)
unrolled_df = unroll_distance_matrix(result_matrix)
print(unrolled_df)

      id_start   id_end  distance
0      1001400  1001402       0.0
1      1001400  1001404       0.0
2      1001400  1001406       0.0
3      1001400  1001408       0.0
4      1001400  1001410       0.0
...        ...      ...       ...
1801   1004356  1001470       0.0
1802   1004356  1001472       0.0
1803   1004356  1001488       0.0
1804   1004356  1004354       0.0
1805   1004356  1004355       0.0

[1806 rows x 3 columns]


In [16]:
def find_ids_within_ten_percentage_threshold(unrolled_df, reference_value):

    reference_df = unrolled_df[unrolled_df['id_start'] == reference_value]


    average_distance = reference_df['distance'].mean()


    threshold_range = 0.1 * average_distance


    ids_within_threshold = unrolled_df[
        (unrolled_df['distance'] >= (average_distance - threshold_range)) &
        (unrolled_df['distance'] <= (average_distance + threshold_range))
    ]['id_start'].unique()


    ids_within_threshold = sorted(ids_within_threshold)

    return ids_within_threshold

In [17]:
reference_value = 1001400
ids_within_threshold = find_ids_within_ten_percentage_threshold(unrolled_df, reference_value)
print(ids_within_threshold)

[1001400, 1001402, 1001404, 1001406, 1001408, 1001410, 1001412, 1001414, 1001416, 1001418, 1001420, 1001422, 1001424, 1001426, 1001428, 1001430, 1001432, 1001434, 1001436, 1001437, 1001438, 1001440, 1001442, 1001444, 1001446, 1001448, 1001450, 1001452, 1001454, 1001456, 1001458, 1001460, 1001461, 1001462, 1001464, 1001466, 1001468, 1001470, 1001472, 1001488, 1004354, 1004355, 1004356]


In [18]:
def calculate_toll_rate(unrolled_df):

    unrolled_df['moto'] = 0.0
    unrolled_df['car'] = 0.0
    unrolled_df['rv'] = 0.0
    unrolled_df['bus'] = 0.0
    unrolled_df['truck'] = 0.0


    rate_coefficients = {'moto': 0.8, 'car': 1.2, 'rv': 1.5, 'bus': 2.2, 'truck': 3.6}


    for vehicle_type, rate_coefficient in rate_coefficients.items():
        unrolled_df[vehicle_type] = unrolled_df['distance'] * rate_coefficient

    return unrolled_df

In [19]:
result_df = calculate_toll_rate(unrolled_df)
print(result_df)

      id_start   id_end  distance  moto  car   rv  bus  truck
0      1001400  1001402       0.0   0.0  0.0  0.0  0.0    0.0
1      1001400  1001404       0.0   0.0  0.0  0.0  0.0    0.0
2      1001400  1001406       0.0   0.0  0.0  0.0  0.0    0.0
3      1001400  1001408       0.0   0.0  0.0  0.0  0.0    0.0
4      1001400  1001410       0.0   0.0  0.0  0.0  0.0    0.0
...        ...      ...       ...   ...  ...  ...  ...    ...
1801   1004356  1001470       0.0   0.0  0.0  0.0  0.0    0.0
1802   1004356  1001472       0.0   0.0  0.0  0.0  0.0    0.0
1803   1004356  1001488       0.0   0.0  0.0  0.0  0.0    0.0
1804   1004356  1004354       0.0   0.0  0.0  0.0  0.0    0.0
1805   1004356  1004355       0.0   0.0  0.0  0.0  0.0    0.0

[1806 rows x 8 columns]


In [40]:
import datetime
def calculate_time_based_toll_rates(df):

    time_ranges = [
        (datetime.time(0, 0, 0), datetime.time(10, 0, 0), 0.8),
        (datetime.time(10, 0, 0), datetime.time(18, 0, 0), 1.2),
        (datetime.time(18, 0, 0), datetime.time(23, 59, 59), 0.8)
    ]


    df['start_day'] = pd.to_datetime(df['start_time']).dt.day_name()
    df['end_day'] = pd.to_datetime(df['end_time']).dt.day_name()
    df['start_time'] = pd.to_datetime(df['start_time']).dt.time
    df['end_time'] = pd.to_datetime(df['end_time']).dt.time


    df['moto_rate'] = 0.0
    df['car_rate'] = 0.0
    df['rv_rate'] = 0.0
    df['bus_rate'] = 0.0
    df['truck_rate'] = 0.0


    for time_range in time_ranges:
        mask = (df['start_time'] >= time_range[0]) & (df['end_time'] <= time_range[1])
        for vehicle_type in ['moto', 'car', 'rv', 'bus', 'truck']:
            df.loc[mask, f'{vehicle_type}_rate'] = df.loc[mask, f'{vehicle_type}_distance'] * time_range[2]

    return df

In [41]:
result_dataframe = calculate_time_based_toll_rates(df3)


print(result_dataframe)

KeyError: ignored